# Replication of SMCalFlow Experiments on V2 Dataset

This notebook explores some of the output from an ONMT model trained on the v2 version of the dataflow dialogues dataset.

In [1]:
from dataflow.core.lispress import *
from dataflow.core.linearize import *
from typeguard import check_type
import json
from pprint import pprint
from onmt.bin.translate import translate
import subprocess

In [2]:
best_checkpoint = "/scratchdata/bking2/tod_as_df_synthesis/output/onmt_models_v2/checkpoint_step_9130.pt"
def simple_infer(utterance: str) -> str:
    options = {"model": best_checkpoint, 
           "max_length": 491, "src": "../output/onmt_text_data/valid.src_tok",
           "replace_unk": True, "n_best": 1, 
           "batch_size": 8, "beam_size": 10, "gpu": 1, "report_time": True, 
           "output": "/tmp/out.nbest"}
    with open("/tmp/out.nbest", "w") as f:
        # clear result first
        f.truncate(0)
    with open("/tmp/utterance.src_tok", "w") as f:
        f.write(utterance)
    options['src'] = "/tmp/utterance.src_tok"
    call: List[str] = ["onmt_translate"]
    for k, v in options.items():
        call.extend([("--" + k), str(v)])
        if type(v) == bool:
            call.pop() # boolean arguments are just flags, no value for them
    print(" ".join(call))
    subprocess.call(call)
    with open("/tmp/out.nbest", "r") as f:
        return parse_lispress(f.read())
print(render_pretty(simple_infer("what is my appointment with janice kang")))

lisp_str: str = """(Yield
  (Event.id
    (singleton
      (QueryEventResponse.results
        (FindEventWrapperWithDefaults
          (Event.attendees_?
            (AttendeeListHasRecipientConstraint
              (RecipientWithNameLike
                (^(Recipient) EmptyStructConstraint)
                (PersonName.apply "janice kang")))))))))
"""
lispress = parse_lispress(lisp_str)


print(render_pretty(lispress))

onmt_translate --model /scratchdata/bking2/tod_as_df_synthesis/output/onmt_models_v2/checkpoint_step_9130.pt --max_length 491 --src /tmp/utterance.src_tok --replace_unk --n_best 1 --batch_size 8 --beam_size 10 --gpu 1 --report_time --output /tmp/out.nbest


[2021-12-10 11:06:24,208 INFO] Translating shard 0.
/soe/bking2/task_oriented_dialogue_as_dataflow_synthesis/venv/lib/python3.7/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


PRED AVG SCORE: -0.0147, PRED PPL: 1.0148
Total translation time (s): 0.214429
Average translation time (s): 0.214429
Tokens per second: 158.560539
(Yield
  (singleton
    (QueryEventResponse.results
      (FindEventWrapperWithDefaults
        (Event.attendees_?
          (AttendeeListHasRecipientConstraint
            (RecipientWithNameLike
              (^(Recipient) EmptyStructConstraint)
              (PersonName.apply " janice "))))))))
(Yield
  (Event.id
    (singleton
      (QueryEventResponse.results
        (FindEventWrapperWithDefaults
          (Event.attendees_?
            (AttendeeListHasRecipientConstraint
              (RecipientWithNameLike
                (^(Recipient) EmptyStructConstraint)
                (PersonName.apply "janice kang")))))))))


In [3]:
simple = "Can you book a meeting at 3pm this afternoon with Matt to discuss status ?"
my_prediction = """
(Yield
  (CreateCommitEventWrapper
    (CreatePreflightEventWrapper
      (&
        (Event.start_? (?= (NextTime (NumberPM 3L))))
        (Event.name_? (?= "discuss status"))
        (Event.attendees_?
          (AttendeeListHasRecipient
            (Execute
              (refer
                (extensionConstraint
                  (RecipientWithNameLike
                    (^(Recipient) EmptyStructConstraint)
                    (PersonName.apply "Matt")))))))))))
"""
print(render_pretty(simple_infer(simple)))


onmt_translate --model /scratchdata/bking2/tod_as_df_synthesis/output/onmt_models_v2/checkpoint_step_9130.pt --max_length 491 --src /tmp/utterance.src_tok --replace_unk --n_best 1 --batch_size 8 --beam_size 10 --gpu 1 --report_time --output /tmp/out.nbest


[2021-12-10 11:06:29,052 INFO] Translating shard 0.
/soe/bking2/task_oriented_dialogue_as_dataflow_synthesis/venv/lib/python3.7/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


PRED AVG SCORE: -0.0038, PRED PPL: 1.0038
Total translation time (s): 0.408814
Average translation time (s): 0.408814
Tokens per second: 166.334831
(Yield
  (CreateCommitEventWrapper
    (CreatePreflightEventWrapper
      (&
        (&
          (Event.subject_? (?= " discuss status "))
          (Event.start_? (DateTimeConstraint (Afternoon) (Today))))
        (Event.attendees_?
          (AttendeeListHasRecipient
            (Execute
              (refer
                (extensionConstraint
                  (RecipientWithNameLike
                    (^(Recipient) EmptyStructConstraint)
                    (PersonName.apply " Matt ")))))))))))


## Results in Brief


In [4]:
with open('../output/evaluation_output_v2/valid.all.scores.json', 'r') as f:
    results = json.loads(f.read())
pprint(results)

{'ignoringRefer': {'accuracy': 0.7276229994072317,
                   'ave_num_turns_before_first_error': 1.757462686567164,
                   'num_correct_dialogues': 1262,
                   'num_correct_turns': 9820,
                   'num_total_dialogues': 3484,
                   'num_total_turns': 13496,
                   'num_turns_before_first_error': 6123,
                   'pct_correct_dialogues': 0.3622273249138921},
 'notIgnoringRefer': {'accuracy': 0.7058387670420866,
                      'ave_num_turns_before_first_error': 1.68398392652124,
                      'num_correct_dialogues': 1183,
                      'num_correct_turns': 9526,
                      'num_total_dialogues': 3484,
                      'num_total_turns': 13496,
                      'num_turns_before_first_error': 5867,
                      'pct_correct_dialogues': 0.33955223880597013}}
